[← Control Systems as Dynamical Systems](../../../getting_started/theory_to_python/control_systems_as_dynamical_systems.rst)

# Example: TurtleBot with Noisy Odometry

Suppose we want to navigate a mobile robot through an environment. We might need it to follow a path, visit waypoints, or track a moving target. To accomplish this, we need accurate knowledge of the robot's position and orientation.

However, our odometry sensor (wheel encoders) is noisy. Uneven terrain, wheel slip, and measurement quantization introduce errors that accumulate over time. In this notebook, we take a classic mobile robotics problem -- a TurtleBot operating with noisy odometry -- and use pykal to recast it as a composition of interacting dynamical systems.

## System Overview

We model the robot dynamics, implement sensor noise, design the observer, and integrate everything into a complete feedback system. This notebook bridges theory and practice for ground robot state estimation.

## Block Diagram

We can model the TurtleBot3 feedback system we are interested in as follows:

![TurtleBot feedback system](../../../_static/tutorial/theory_to_python/turtlebot_feedback_system.svg)

where **waypoint generator** produces reference trajectories (e.g., move to coordinates), **velocity command** is a high-level controller that outputs linear and angular velocity, **TurtleBot** is our differential-drive robot (our plant), **odometry sensor** is the wheel encoders measuring position and orientation, and **Kalman Filter** is the state observer that fuses control inputs and noisy measurements.

:::{note}
The TurtleBot has an onboard low-level controller that converts velocity commands to wheel speeds. Our "controller", then, is simply the high-level velocity command generator.
:::

### Discrete-time Dynamical Systems

We cast each component as a discrete-time dynamical system:

![TurtleBot as composition of dynamical systems](../../../_static/tutorial/theory_to_python/turtlebot_composition_of_dynamical_systems.svg)

We discuss the derivation of each dynamical system block below.

:::{note}
Unlike the cruise control example, the TurtleBot has **nonlinear** kinematics (due to the orientation-dependent motion). This makes the Kalman filter an Extended Kalman Filter (EKF), linearizing around the current estimate at each step.
:::

### Block 1: Waypoint Generator

<div style="text-align: center;">
 <img src="../../../_static/tutorial/theory_to_python/turtlebot_waypoint_block.svg"
      style="max-width: 100%;">
</div>

The waypoint generator produces reference poses $(x_r, y_r, \theta_r)$ for the robot to track. For this tutorial, we'll create a simple square trajectory.

**State**: Current waypoint index $w_k \in \{0, 1, 2, 3\}$

**Evolution**:

$$
w_{k+1} = (w_k + 1) \mod 4 \quad \text{when waypoint reached}
$$

**Output**: Reference pose

$$
r_k = \begin{bmatrix} x_r \\ y_r \\ \theta_r \end{bmatrix}
$$

For now, we'll define the waypoints as a simple list. Later, we'll integrate this into a stateful system.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pykal import DynamicalSystem
from pykal.data_change import corrupt, prepare  # For realistic sensor noise


# Create square trajectory: (x, y, theta) in meters and radians
square_waypoints = [
    (2.0, 0.0, 0.0),  # Right
    (2.0, 2.0, np.pi / 2),  # Up
    (0.0, 2.0, np.pi),  # Left
    (0.0, 0.0, -np.pi / 2),  # Down (back to start)
]


# Simple helper function to get waypoint by index
def get_waypoint(waypoints, idx):
    """Get waypoint as numpy array."""
    x_r, y_r, theta_r = waypoints[idx % len(waypoints)]
    return np.array([[x_r], [y_r], [theta_r]])


# Test getting waypoints
print("Waypoint 0:", get_waypoint(square_waypoints, 0).flatten())
print("Waypoint 1:", get_waypoint(square_waypoints, 1).flatten())
print("Waypoint 2:", get_waypoint(square_waypoints, 2).flatten())

### Block 2: Velocity Command Generator

<div style="text-align: center;">
 <img src="../../../_static/tutorial/theory_to_python/turtlebot_controller_block.svg"
      style="max-width: 100%;">
</div>

The velocity controller generates $(v_{cmd}, \omega_{cmd})$ -- linear and angular velocity commands -- to drive the robot toward the reference pose.

We use a simple proportional controller:

**Inputs**: Current estimate $\hat{x} = [\hat{x}, \hat{y}, \hat{\theta}]^T$, reference $r = [x_r, y_r, \theta_r]^T$

**Outputs**:

$$
v_{cmd} = K_v \cdot d \quad \text{(linear velocity proportional to distance)}
$$

$$
\omega_{cmd} = K_\omega \cdot (\theta_r - \hat{\theta}) \quad \text{(angular velocity proportional to heading error)}
$$

where $d = \sqrt{(x_r - \hat{x})^2 + (y_r - \hat{y})^2}$

This controller is **stateless** (no internal dynamics), so we implement it as a pure function:


In [ ]:
def velocity_controller(
    xhat: np.ndarray,  # Current estimate [x, y, theta]
    r: np.ndarray,  # Reference [x_r, y_r, theta_r]
    Kv: float = 0.5,
    Komega: float = 1.0,
    max_v: float = 0.22,  # TurtleBot max linear velocity (m/s)
    max_omega: float = 2.84,  # TurtleBot max angular velocity (rad/s)
) -> np.ndarray:
    """
    Proportional controller for TurtleBot velocity commands.

    Returns
    -------
    cmd : np.ndarray
        Shape (2,1) containing [v_cmd, omega_cmd]
    """
    # Extract positions
    x, y, theta = xhat.flatten()[:3]
    x_r, y_r, theta_r = r.flatten()

    # Distance to goal
    dx = x_r - x
    dy = y_r - y
    distance = np.sqrt(dx**2 + dy**2)

    # Heading error (wrap to [-pi, pi])
    heading_error = theta_r - theta
    heading_error = np.arctan2(np.sin(heading_error), np.cos(heading_error))

    # Proportional control
    v_cmd = Kv * distance
    omega_cmd = Komega * heading_error

    # Saturate commands
    v_cmd = np.clip(v_cmd, -max_v, max_v)
    omega_cmd = np.clip(omega_cmd, -max_omega, max_omega)

    return np.array([[v_cmd], [omega_cmd]])


# Test the controller
xhat_test = np.array([[0.0], [0.0], [0.0]])
r_test = np.array([[1.0], [1.0], [np.pi / 4]])
cmd = velocity_controller(xhat_test, r_test)
print(f"Velocity command: v={cmd[0,0]:.3f} m/s, omega={cmd[1,0]:.3f} rad/s")

### Block 3: TurtleBot (Plant)

<div style="text-align: center;">
 <img src="../../../_static/tutorial/theory_to_python/turtlebot_plant_block.svg"
      style="max-width: 100%;">
</div>

The TurtleBot is a differential-drive mobile robot. Its motion follows the **unicycle kinematics model**:

**State**:

$$
x_k = \begin{bmatrix} x \\ y \\ \theta \\ v \\ \omega \end{bmatrix}
$$

where $(x, y)$ is position, $\theta$ is heading, $v$ is linear velocity, $\omega$ is angular velocity.

**Inputs**: Velocity commands

$$
u_k = \begin{bmatrix} v_{cmd} \\ \omega_{cmd} \end{bmatrix}
$$

**Discrete-time dynamics** (using Euler integration with timestep $\Delta t$):

$$

\begin{aligned}
x_{k+1} &= x_k + v_k \cos(\theta_k) \Delta t \\
y_{k+1} &= y_k + v_k \sin(\theta_k) \Delta t \\
\theta_{k+1} &= \theta_k + \omega_k \Delta t \\
v_{k+1} &= v_{cmd} \quad \text{(assume instantaneous velocity response)} \\
\omega_{k+1} &= \omega_{cmd}
\end{aligned}

$$

We wrap $\theta$ to $[-\pi, \pi]$ to avoid angle wrapping issues.

**Measurement model**: We observe $[x, y, \theta]$ from wheel odometry (noisy).

$$
y_k = \begin{bmatrix} x_k \\ y_k \\ \theta_k \end{bmatrix} + v_k
$$

where $v_k \sim \mathcal{N}(0, R)$ is measurement noise.

We implement this discrete-time dynamical system below.


In [ ]:
def turtlebot_f(xk: np.ndarray, uk: np.ndarray, dt: float) -> np.ndarray:
    """
    TurtleBot unicycle dynamics (noise-free).

    Parameters
    ----------
    xk : np.ndarray
        Current state [x, y, theta, v, omega], shape (5,1)
    uk : np.ndarray
        Control input [v_cmd, omega_cmd], shape (2,1)
    dt : float
        Timestep (seconds)

    Returns
    -------
    xk_next : np.ndarray
        Next state, shape (5,1)
    """
    x, y, theta, v, omega = xk.flatten()
    v_cmd, omega_cmd = uk.flatten()

    # Euler integration
    x_new = x + v * np.cos(theta) * dt
    y_new = y + v * np.sin(theta) * dt
    theta_new = theta + omega * dt

    # Wrap theta to [-pi, pi]
    theta_new = np.arctan2(np.sin(theta_new), np.cos(theta_new))

    # Update velocities (assume instantaneous response)
    v_new = v_cmd
    omega_new = omega_cmd

    return np.array([[x_new], [y_new], [theta_new], [v_new], [omega_new]])


def turtlebot_h(xk: np.ndarray) -> np.ndarray:
    """
    Measurement function: observe [x, y, theta] from odometry.

    Parameters
    ----------
    xk : np.ndarray
        State [x, y, theta, v, omega], shape (5,1)

    Returns
    -------
    yk : np.ndarray
        Measurement [x, y, theta], shape (3,1)
    """
    return xk[:3, :]


# Create the plant DynamicalSystem
plant = DynamicalSystem(f=turtlebot_f, h=turtlebot_h, state_name="xk")

# Test the plant
xk_test = np.array([[0.0], [0.0], [0.0], [0.0], [0.0]])  # Start at origin, stationary
uk_test = np.array([[0.2], [0.5]])  # Move forward and turn
dt = 0.1

xk_next, yk = plant.step(
    param_dict={"xk": xk_test, "uk": uk_test, "dt": dt}, return_state=True
)
print("After one step:")
print(f"  State: {xk_next.flatten()}")
print(f"  Measurement: {yk.flatten()}")

### Block 4: Kalman Filter (Observer)

<div style="text-align: center;">
 <img src="../../../_static/tutorial/theory_to_python/turtlebot_observer_block.svg"
      style="max-width: 100%;">
</div>

The Kalman filter estimates the TurtleBot's state $\hat{x}_k$ by fusing:
1. **Prediction**: Motion model propagates state forward using control inputs
2. **Update**: Noisy odometry measurements correct the prediction

Since the dynamics are **nonlinear** (due to $\cos(\theta)$ and $\sin(\theta)$), we use the **Extended Kalman Filter (EKF)**, which linearizes around the current estimate.

**Jacobian of dynamics** (for covariance propagation):

$$

F_k = \frac{\partial f}{\partial x}\bigg|_{\hat{x}_k} =
\begin{bmatrix}
1 & 0 & -v \sin(\theta) \Delta t & \cos(\theta) \Delta t & 0 \\
0 & 1 & v \cos(\theta) \Delta t & \sin(\theta) \Delta t & 0 \\
0 & 0 & 1 & 0 & \Delta t \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1
\end{bmatrix}

$$

**Jacobian of measurement**:

$$

H_k = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0
\end{bmatrix}

$$

**Noise covariances**:
- Process noise $Q$: Uncertainty in velocity model
- Measurement noise $R$: Wheel odometry error

The implementation can be found in [KF](../../../src/pykal/algorithm_library/estimators/kf.py).


In [ ]:
from pykal.algorithm_library.estimators import kf


def compute_F_jacobian(xhat: np.ndarray, dt: float) -> np.ndarray:
    """
    Compute Jacobian of dynamics for TurtleBot.

    Parameters
    ----------
    xhat : np.ndarray
        Current state estimate [x, y, theta, v, omega], shape (5,1)
    dt : float
        Timestep

    Returns
    -------
    F : np.ndarray
        Jacobian matrix, shape (5, 5)
    """
    _, _, theta, v, _ = xhat.flatten()

    F = np.array(
        [
            [1, 0, -v * np.sin(theta) * dt, np.cos(theta) * dt, 0],
            [0, 1, v * np.cos(theta) * dt, np.sin(theta) * dt, 0],
            [0, 0, 1, 0, dt],
            [0, 0, 0, 1, 0],
            [0, 0, 0, 0, 1],
        ]
    )

    return F


def compute_H_jacobian() -> np.ndarray:
    """
    Compute Jacobian of measurement function (constant for this system).

    Returns
    -------
    H : np.ndarray
        Jacobian matrix, shape (3, 5)
    """
    return np.array([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0]])


# Process and measurement noise covariances
Q_turtlebot = np.diag([0.01, 0.01, 0.02, 0.1, 0.1])  # Process noise
R_turtlebot = np.diag([0.05, 0.05, 0.1])  # Odometry noise (5cm, 5cm, ~6 degrees)

# Create the observer DynamicalSystem
observer = DynamicalSystem(f=kf.f, h=kf.h, state_name="xhat_P")

# Test the observer (single step)
xhat_0 = np.array([[0.0], [0.0], [0.0], [0.0], [0.0]])
P_0 = np.diag([0.1, 0.1, 0.1, 1.0, 1.0])
xhat_P_test = (xhat_0, P_0)

# Simulate receiving a noisy measurement
yk_noisy = np.array([[0.02], [0.01], [0.05]])  # Noisy measurement

xhat_P_new, xhat_out = observer.step(
    param_dict={
        "xhat_P": xhat_P_test,
        "yk": yk_noisy,
        "f": turtlebot_f,
        "f_params": {"xk": xhat_0, "uk": np.array([[0.0], [0.0]]), "dt": 0.1},
        "h": turtlebot_h,
        "h_params": {"xk": xhat_0},
        "Fk": compute_F_jacobian(xhat_0, dt=0.1),
        "Qk": Q_turtlebot,
        "Hk": compute_H_jacobian(),
        "Rk": R_turtlebot,
    },
    return_state=True,
)

print("Observer estimate after one measurement:")
print(xhat_out.flatten())

## Simulation

![Complete TurtleBot system](../../../_static/tutorial/theory_to_python/turtlebot_composition_of_dynamical_systems.svg)

We now simulate the complete closed-loop system, integrating all four dynamical components:
1. **Waypoint Generator** → reference pose $r_k$
2. **Velocity Controller** → velocity commands $u_k$ (using $r_k$ and $\hat{x}_k$)
3. **TurtleBot Plant** → true state evolution and noisy measurements $y_k$
4. **Kalman Filter** → state estimate $\hat{x}_k$ (using $u_k$ and $y_k$)

### System Parameters

In [ ]:
# Time parameters
dt = 0.1  # Sampling time (seconds)
switch_time = 15.0  # Time at each waypoint (seconds)

# Controller gains
Kv = 0.5  # Linear velocity gain
Komega = 1.5  # Angular velocity gain

# Kalman filter parameters
Q = np.diag([0.01, 0.01, 0.02, 0.1, 0.1])  # Process noise covariance
R = np.diag([0.05, 0.05, 0.1])  # Measurement noise covariance

### Initial Conditions

In [ ]:
# Initial states
waypoint_idx = 0  # Start at first waypoint
time_at_waypoint = 0.0  # Time spent at current waypoint

xk = np.array([[0.0], [0.0], [0.0], [0.0], [0.0]])  # Plant state: start at origin
xhat = np.array([[0.0], [0.0], [0.0], [0.0], [0.0]])  # Observer estimate
P = np.diag([0.1, 0.1, 0.1, 1.0, 1.0])  # Covariance matrix
xhat_P = (xhat, P)  # Observer state tuple

# Storage for plotting
time_hist = []
reference_hist = []
true_state_hist = []
estimate_hist = []
measurement_hist = []
command_hist = []
error_hist = []

### Simulate

In [ ]:
# Simulation time
T_sim = 60.0  # seconds
time_steps = np.arange(0, T_sim, dt)

# Run closed-loop simulation
for tk in time_steps:
    # 1. Waypoint generator (simple switching logic)
    time_at_waypoint += dt
    if time_at_waypoint >= switch_time:
        waypoint_idx = (waypoint_idx + 1) % len(square_waypoints)
        time_at_waypoint = 0.0

    rk = get_waypoint(square_waypoints, waypoint_idx)

    # 2. Extract current state estimate from observer
    xhat = observer.h(xhat_P)

    # 3. Velocity controller step
    uk = velocity_controller(xhat, rk, Kv=Kv, Komega=Komega)

    # 4. Plant step (true dynamics)
    xk, yk_clean = plant.step(
        return_state=True, param_dict={"xk": xk, "uk": uk, "dt": dt}
    )

    # 5. Add measurement noise (realistic corruption)
    yk_flat = yk_clean.flatten()

    # X: bias (wheel diameter mismatch) + Gaussian noise
    yk_corrupted_x = corrupt.with_bias(yk_flat[0], bias=0.02)
    yk_corrupted_x = yk_corrupted_x + np.random.normal(0, np.sqrt(R[0, 0]))

    # Y: Gaussian noise
    yk_corrupted_y = yk_flat[1] + np.random.normal(0, np.sqrt(R[1, 1]))

    # Theta: Gaussian noise + occasional spikes (bumps)
    yk_corrupted_theta = yk_flat[2] + np.random.normal(0, np.sqrt(R[2, 2]))
    if np.random.rand() < 0.02:  # 2% spike rate
        yk_corrupted_theta += np.random.choice([-1, 1]) * 0.3

    yk_noisy = np.array([[yk_corrupted_x], [yk_corrupted_y], [yk_corrupted_theta]])

    # 6. Observer step (EKF)
    Fk = compute_F_jacobian(xhat, dt)
    Hk = compute_H_jacobian()

    xhat_P, xhat_obs = observer.step(
        return_state=True,
        param_dict={
            "xhat_P": xhat_P,
            "yk": yk_noisy,
            "f": turtlebot_f,
            "f_params": {"xk": xhat, "uk": uk, "dt": dt},
            "h": turtlebot_h,
            "h_params": {"xk": xhat},
            "Fk": Fk,
            "Qk": Q,
            "Hk": Hk,
            "Rk": R,
        },
    )

    # Store results
    time_hist.append(tk)
    reference_hist.append(rk.flatten())
    true_state_hist.append(xk.flatten())
    estimate_hist.append(xhat.flatten())
    measurement_hist.append(yk_noisy.flatten())
    command_hist.append(uk.flatten())
    error_hist.append((xk - xhat).flatten())

# Convert to numpy arrays
reference_hist = np.array(reference_hist)
true_state_hist = np.array(true_state_hist)
estimate_hist = np.array(estimate_hist)
measurement_hist = np.array(measurement_hist)
command_hist = np.array(command_hist)
error_hist = np.array(error_hist)

### Visualize

We can visualize the pertinent values of our system to assure correct behavior. Note how the Kalman filter smooths out the noisy measurements and provides accurate state estimates even in the presence of sensor corruption.

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(14, 12))

# Plot 1: 2D Trajectory
ax = axs[0, 0]
ax.plot(
    true_state_hist[:, 0],
    true_state_hist[:, 1],
    "b-",
    label="True Trajectory",
    linewidth=2,
    alpha=0.7,
)
ax.plot(
    estimate_hist[:, 0],
    estimate_hist[:, 1],
    "r--",
    label="Estimated Trajectory",
    linewidth=2,
    alpha=0.7,
)
ax.scatter(
    reference_hist[:, 0],
    reference_hist[:, 1],
    c="green",
    s=100,
    marker="*",
    label="Waypoints",
    zorder=5,
)
ax.set_xlabel("X Position (m)", fontsize=11)
ax.set_ylabel("Y Position (m)", fontsize=11)
ax.set_title("2D Trajectory Tracking", fontsize=13, fontweight="bold")
ax.legend()
ax.grid(True, alpha=0.3)
ax.axis("equal")

# Plot 2: X Position vs Time
ax = axs[0, 1]
ax.plot(time_hist, true_state_hist[:, 0], "b-", label="True X", alpha=0.7)
ax.plot(time_hist, estimate_hist[:, 0], "r--", label="Estimated X", alpha=0.7)
ax.scatter(
    time_hist[::10],
    measurement_hist[::10, 0],
    c="gray",
    s=10,
    alpha=0.3,
    label="Measurements",
)
ax.set_ylabel("X Position (m)", fontsize=11)
ax.set_title("X Position Over Time", fontsize=13, fontweight="bold")
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Y Position vs Time
ax = axs[1, 0]
ax.plot(time_hist, true_state_hist[:, 1], "b-", label="True Y", alpha=0.7)
ax.plot(time_hist, estimate_hist[:, 1], "r--", label="Estimated Y", alpha=0.7)
ax.scatter(
    time_hist[::10],
    measurement_hist[::10, 1],
    c="gray",
    s=10,
    alpha=0.3,
    label="Measurements",
)
ax.set_ylabel("Y Position (m)", fontsize=11)
ax.set_title("Y Position Over Time", fontsize=13, fontweight="bold")
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Heading vs Time
ax = axs[1, 1]
ax.plot(
    time_hist,
    np.rad2deg(true_state_hist[:, 2]),
    "b-",
    label="True Heading",
    alpha=0.7,
)
ax.plot(
    time_hist,
    np.rad2deg(estimate_hist[:, 2]),
    "r--",
    label="Estimated Heading",
    alpha=0.7,
)
ax.set_ylabel("Heading (degrees)", fontsize=11)
ax.set_title("Heading Over Time", fontsize=13, fontweight="bold")
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 5: Velocity Commands
ax = axs[2, 0]
ax.plot(
    time_hist,
    command_hist[:, 0],
    "g-",
    label="Linear Velocity",
    linewidth=1.5,
)
ax.set_ylabel("Linear Velocity (m/s)", fontsize=11)
ax.set_xlabel("Time (s)", fontsize=11)
ax.set_title("Control Commands", fontsize=13, fontweight="bold")
ax.legend(loc="upper left")
ax.grid(True, alpha=0.3)

ax_omega = ax.twinx()
ax_omega.plot(
    time_hist,
    command_hist[:, 1],
    "purple",
    label="Angular Velocity",
    linewidth=1.5,
    alpha=0.7,
)
ax_omega.set_ylabel("Angular Velocity (rad/s)", fontsize=11, color="purple")
ax_omega.tick_params(axis="y", labelcolor="purple")
ax_omega.legend(loc="upper right")

# Plot 6: Position Estimation Error
ax = axs[2, 1]
position_error = np.sqrt(error_hist[:, 0] ** 2 + error_hist[:, 1] ** 2)
ax.plot(time_hist, position_error, "m-", label="Position Error", linewidth=1.5)
ax.set_xlabel("Time (s)", fontsize=11)
ax.set_ylabel("Position Error (m)", fontsize=11)
ax.set_title("Estimation Error Magnitude", fontsize=13, fontweight="bold")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean position error: {np.mean(position_error):.4f} m")
print(f"Max position error: {np.max(position_error):.4f} m")

## Callback Wrapper

Just as we did with the cruise control system, we can wrap the **entire TurtleBot navigation system** in a single callback. This encapsulates all four dynamical systems (waypoint generator, velocity controller, plant, and observer) and their states, making it easy to reuse and experiment with different configurations. This pattern is particularly useful for integration with ROS2, as we'll see in later tutorials.

In [ ]:
def initialize_waypoint_generator(waypoints: list, switch_time: float = 5.0):
    """
    Returns a stateful callback that cycles through waypoints.

    Parameters
    ----------
    waypoints : list of tuples
        Each tuple is (x, y, theta) representing target pose
    switch_time : float
        Time to spend at each waypoint before switching
    """
    current_idx = 0
    time_at_waypoint = 0.0

    def callback(tk: float, dt: float) -> np.ndarray:
        nonlocal current_idx, time_at_waypoint

        time_at_waypoint += dt

        if time_at_waypoint >= switch_time:
            current_idx = (current_idx + 1) % len(waypoints)
            time_at_waypoint = 0.0

        x_r, y_r, theta_r = waypoints[current_idx]
        return np.array([[x_r], [y_r], [theta_r]])

    return callback


def initialize_turtlebot_system(
    # Initial states
    xk_init: np.ndarray = None,
    xhat_init: np.ndarray = None,
    P_init: np.ndarray = None,
    # System parameters
    dt: float = 0.1,
    Kv: float = 0.5,
    Komega: float = 1.0,
    # Noise parameters
    Q: np.ndarray = None,
    R: np.ndarray = None,
    # Waypoint parameters
    waypoints: list = None,
    switch_time: float = 10.0,
    # Realistic corruption parameters
    use_realistic_corruption: bool = True,
    bias_x: float = 0.02,  # 2cm systematic offset
    spike_rate: float = 0.02,  # 2% of measurements
    spike_magnitude: float = 0.3,  # ~17 degrees
):
    """
    Initialize complete TurtleBot navigation system with realistic sensor corruption.

    Returns a stateful callback that executes one timestep of the full system.

    Parameters
    ----------
    use_realistic_corruption : bool
        If True, uses data_change methods for realistic corruption.
        If False, uses simple Gaussian noise.
    bias_x : float
        Systematic bias for X odometry (wheel diameter mismatch)
    spike_rate : float
        Probability of spike in heading measurement (bumps/obstacles)
    spike_magnitude : float
        Magnitude of heading spikes in radians
    """
    # Default initialization
    if xk_init is None:
        xk_init = np.array([[0.0], [0.0], [0.0], [0.0], [0.0]])
    if xhat_init is None:
        xhat_init = np.array([[0.0], [0.0], [0.0], [0.0], [0.0]])
    if P_init is None:
        P_init = np.diag([0.1, 0.1, 0.1, 1.0, 1.0])
    if Q is None:
        Q = np.diag([0.01, 0.01, 0.02, 0.1, 0.1])
    if R is None:
        R = np.diag([0.05, 0.05, 0.1])
    if waypoints is None:
        waypoints = [
            (2.0, 0.0, 0.0),
            (2.0, 2.0, np.pi / 2),
            (0.0, 2.0, np.pi),
            (0.0, 0.0, -np.pi / 2),
        ]

    # Initialize internal states
    xk = xk_init.copy()
    xhat_P = (xhat_init.copy(), P_init.copy())

    # Initialize waypoint generator
    waypoint_gen = initialize_waypoint_generator(waypoints, switch_time)

    def callback(tk: float) -> dict:
        nonlocal xk, xhat_P

        # 1. Get current waypoint
        rk = waypoint_gen(tk, dt)

        # 2. Extract current estimate
        xhat = KF.h(xhat_P)

        # 3. Generate velocity command
        uk = velocity_controller(xhat, rk, Kv=Kv, Komega=Komega)

        # 4. Plant step (true dynamics)
        xk, yk_clean = turtlebot_plant.step(
            param_dict={"xk": xk, "uk": uk, "dt": dt}, return_state=True
        )

        # 5. Add measurement corruption
        yk_flat = yk_clean.flatten()

        if use_realistic_corruption:
            # Realistic odometry corruption pipeline using data_change
            # X: bias (wheel diameter mismatch) + Gaussian noise
            yk_corrupted_x = corrupt.with_bias(yk_flat[0], bias=bias_x)
            yk_corrupted_x = yk_corrupted_x + np.random.normal(0, np.sqrt(R[0, 0]))

            # Y: Gaussian noise (no systematic bias)
            yk_corrupted_y = yk_flat[1] + np.random.normal(0, np.sqrt(R[1, 1]))

            # Theta: Gaussian noise + occasional spikes (bumps/obstacles)
            yk_corrupted_theta = yk_flat[2] + np.random.normal(0, np.sqrt(R[2, 2]))
            if np.random.rand() < spike_rate:
                yk_corrupted_theta += np.random.choice([-1, 1]) * spike_magnitude

            yk_noisy = np.array(
                [[yk_corrupted_x], [yk_corrupted_y], [yk_corrupted_theta]]
            )
        else:
            # Simple Gaussian noise (original method)
            yk_noisy = yk_clean + np.random.multivariate_normal(np.zeros(3), R).reshape(
                -1, 1
            )

        # 6. Observer step (EKF)
        Fk = compute_F_jacobian(xhat, dt)
        Hk = compute_H_jacobian()

        xhat_P, xhat = turtlebot_observer.step(
            param_dict={
                "xhat_P": xhat_P,
                "yk": yk_noisy,
                "f": turtlebot_f,
                "f_params": {"xk": xhat, "uk": uk, "dt": dt},
                "h": turtlebot_h,
                "h_params": {"xk": xhat},
                "Fk": Fk,
                "Qk": Q,
                "Hk": Hk,
                "Rk": R,
            },
            return_state=True,
        )

        # Return all relevant signals
        return {
            "time": tk,
            "reference": rk.flatten(),
            "command": uk.flatten(),
            "true_state": xk.flatten(),
            "measurement": yk_noisy.flatten(),
            "estimate": xhat.flatten(),
            "estimation_error": (xk - xhat).flatten(),
        }

    return callback

Now we can run the same simulation with a much cleaner interface. All system parameters can be configured at initialization, and the simulation loop becomes trivial:

In [ ]:
# Initialize the complete TurtleBot navigation system
np.random.seed(42)
turtlebot_system = initialize_turtlebot_system(
    # Initial states
    xk_init=np.array([[0.0], [0.0], [0.0], [0.0], [0.0]]),
    xhat_init=np.array([[0.0], [0.0], [0.0], [0.0], [0.0]]),
    P_init=np.diag([0.1, 0.1, 0.1, 1.0, 1.0]),
    # System parameters
    dt=0.1,
    Kv=0.5,
    Komega=1.5,
    # Kalman filter parameters
    Q=np.diag([0.01, 0.01, 0.02, 0.1, 0.1]),
    R=np.diag([0.05, 0.05, 0.1]),
    # Waypoint parameters
    waypoints=[
        (2.0, 0.0, 0.0),
        (2.0, 2.0, np.pi / 2),
        (0.0, 2.0, np.pi),
        (0.0, 0.0, -np.pi / 2),
    ],
    switch_time=15.0,
    # Realistic corruption
    use_realistic_corruption=True,
)

# Simulation time
T_sim = 60.0
dt = 0.1
time_steps = np.arange(0, T_sim, dt)

# Storage for results
results = {
    "time": [],
    "reference": [],
    "command": [],
    "true_state": [],
    "measurement": [],
    "estimate": [],
    "estimation_error": [],
}

# Run simulation
for tk in time_steps:
    output = turtlebot_system(tk)

    results["time"].append(output["time"])
    results["reference"].append(output["reference"])
    results["command"].append(output["command"])
    results["true_state"].append(output["true_state"])
    results["measurement"].append(output["measurement"])
    results["estimate"].append(output["estimate"])
    results["estimation_error"].append(output["estimation_error"])

# Convert to numpy arrays for plotting
for key in [
    "reference",
    "command",
    "true_state",
    "measurement",
    "estimate",
    "estimation_error",
]:
    results[key] = np.array(results[key])

In [ ]:
# Visualize results
fig, axs = plt.subplots(3, 2, figsize=(14, 12))

# Plot 1: 2D Trajectory
ax = axs[0, 0]
ax.plot(
    results["true_state"][:, 0],
    results["true_state"][:, 1],
    "b-",
    label="True Trajectory",
    linewidth=2,
    alpha=0.7,
)
ax.plot(
    results["estimate"][:, 0],
    results["estimate"][:, 1],
    "r--",
    label="Estimated Trajectory",
    linewidth=2,
    alpha=0.7,
)
ax.scatter(
    results["reference"][:, 0],
    results["reference"][:, 1],
    c="green",
    s=100,
    marker="*",
    label="Waypoints",
    zorder=5,
)
ax.set_xlabel("X Position (m)", fontsize=11)
ax.set_ylabel("Y Position (m)", fontsize=11)
ax.set_title(
    "2D Trajectory Tracking (Callback Interface)", fontsize=13, fontweight="bold"
)
ax.legend()
ax.grid(True, alpha=0.3)
ax.axis("equal")

# Plot 2: X Position vs Time
ax = axs[0, 1]
ax.plot(
    results["time"],
    results["true_state"][:, 0],
    "b-",
    label="True X",
    alpha=0.7,
)
ax.plot(
    results["time"],
    results["estimate"][:, 0],
    "r--",
    label="Estimated X",
    alpha=0.7,
)
ax.scatter(
    results["time"][::10],
    results["measurement"][::10, 0],
    c="gray",
    s=10,
    alpha=0.3,
    label="Measurements",
)
ax.set_ylabel("X Position (m)", fontsize=11)
ax.set_title("X Position Over Time", fontsize=13, fontweight="bold")
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Y Position vs Time
ax = axs[1, 0]
ax.plot(
    results["time"],
    results["true_state"][:, 1],
    "b-",
    label="True Y",
    alpha=0.7,
)
ax.plot(
    results["time"],
    results["estimate"][:, 1],
    "r--",
    label="Estimated Y",
    alpha=0.7,
)
ax.scatter(
    results["time"][::10],
    results["measurement"][::10, 1],
    c="gray",
    s=10,
    alpha=0.3,
    label="Measurements",
)
ax.set_ylabel("Y Position (m)", fontsize=11)
ax.set_title("Y Position Over Time", fontsize=13, fontweight="bold")
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Heading vs Time
ax = axs[1, 1]
ax.plot(
    results["time"],
    np.rad2deg(results["true_state"][:, 2]),
    "b-",
    label="True Heading",
    alpha=0.7,
)
ax.plot(
    results["time"],
    np.rad2deg(results["estimate"][:, 2]),
    "r--",
    label="Estimated Heading",
    alpha=0.7,
)
ax.set_ylabel("Heading (degrees)", fontsize=11)
ax.set_title("Heading Over Time", fontsize=13, fontweight="bold")
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 5: Velocity Commands
ax = axs[2, 0]
ax.plot(
    results["time"],
    results["command"][:, 0],
    "g-",
    label="Linear Velocity",
    linewidth=1.5,
)
ax.set_ylabel("Linear Velocity (m/s)", fontsize=11)
ax.set_xlabel("Time (s)", fontsize=11)
ax.set_title("Control Commands", fontsize=13, fontweight="bold")
ax.legend(loc="upper left")
ax.grid(True, alpha=0.3)

ax_omega = ax.twinx()
ax_omega.plot(
    results["time"],
    results["command"][:, 1],
    "purple",
    label="Angular Velocity",
    linewidth=1.5,
    alpha=0.7,
)
ax_omega.set_ylabel("Angular Velocity (rad/s)", fontsize=11, color="purple")
ax_omega.tick_params(axis="y", labelcolor="purple")
ax_omega.legend(loc="upper right")

# Plot 6: Position Estimation Error
ax = axs[2, 1]
position_error = np.sqrt(
    results["estimation_error"][:, 0] ** 2 + results["estimation_error"][:, 1] ** 2
)
ax.plot(
    results["time"],
    position_error,
    "m-",
    label="Position Error",
    linewidth=1.5,
)
ax.set_xlabel("Time (s)", fontsize=11)
ax.set_ylabel("Position Error (m)", fontsize=11)
ax.set_title("Estimation Error Magnitude", fontsize=13, fontweight="bold")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean position error: {np.mean(position_error):.4f} m")
print(f"Max position error: {np.max(position_error):.4f} m")

## Experimentation

Now that we have a working system, try experimenting with different parameters:

**Noise tuning**:
- Increase measurement noise `R` to simulate worse odometry (wheel slip, rough terrain)
- Increase process noise `Q` to account for model uncertainty
- Observe how the filter trades off model prediction vs measurements

**Controller tuning**:
- Adjust `Kv` and `Komega` to change aggressiveness
- Try different waypoint patterns (figure-eight, circle, random)
- Add velocity limits to simulate real robot constraints

**Observer comparison**:
- Run simulation with KF disabled (use raw measurements)
- Compare estimation error with and without filtering
- Visualize covariance ellipses over time

**Challenge**: Can you tune Q and R to minimize position error while maintaining smooth estimates?

[← Control Systems as Dynamical Systems](../../../getting_started/theory_to_python/control_systems_as_dynamical_systems.rst)